In [2]:
from requests import get, Response
from bs4 import BeautifulSoup
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data = pd.DataFrame(columns=['department', 'code', 'name', 'description', 'credit', 'restrictions'])

In [6]:
def get_soup(url: str) -> BeautifulSoup:
    response: Response = get(url)
    return BeautifulSoup(response.content, "lxml")


def get_course_list(soup: BeautifulSoup) -> list[str]:
    names = soup.find_all(class_="course-name")
    descriptions = soup.find_all(class_="course-descriptions")
    print(break_course_name(names[0].text))
    print(break_course_desc(descriptions[0].text))
    course_info = [
        (name.text + "\n" + description.text).strip()
        for name, description in list(zip(names, descriptions))
    ]
    return course_info

def break_course_name(course: str) -> tuple[str, str, str]:
    code = course.split('.')[0]
    name = course.split('.')[1].split('(')[0]
    credit = '(' + course.split('(')[1]
    return code, name, credit

def break_course_desc(course: str) -> tuple[str, str]:     
    desc = course.split('Prerequisites:')
    return desc[0], desc[1] if len(desc)>1 else ''


def get_catalog(url: str, domain: str = "catalog") -> list[str]:
    visited: set[str] = set()
    soup: BeautifulSoup = get_soup(url)

    visited.update(
        [
            f'https://catalog.ucsd.edu{link["href"][2:]}'
            for link in soup.find_all("a", href=True)
            if "../courses" in link["href"]
        ]
    )

    return list(visited)


def get_all_courses() -> dict[str, list[str]]:
    catalog = get_catalog("https://catalog.ucsd.edu/front/courses.html")
    courses: dict[str, list[str]] = dict()
    for subject in catalog:
        soup = get_soup(subject)
        courses[subject.split("/")[-1][:-5]] = get_course_list(soup)
    return courses

In [7]:
get_all_courses()

('MUS 1A', ' Fundamentals of Music A ', '(4)')
('This course, first in a three-quarter sequence, is primarily intended for students without previous musical experience. It introduces music notation and basic music theory topics such as intervals, scales, keys, and chords, as well as basic rhythm skills. ', ' none.')
('DOC 1', ' Dimensions of Culture: Diversity ', '(4)')
('This course focuses on sociocultural diversity in examining class, ethnicity, race, gender, and sexuality as significant markers of differences among persons. Emphasizing American society while cultivating students’ critical reading skills, it explores the cultural understandings of diversity and its economic, moral, and political consequences. Restrictions: Marshall College students only, must be taken for a letter grade. ', ' satisfaction of UC Entry Level Writing Requirement or concurrent enrollment in AWP 3, Analytical Writing. (F)')
('HUM 1', ' The Foundations of Western Civilization: Israel and Greece ', '(6)')


KeyboardInterrupt: 